In [5]:
import os
from typing import List
from neo4j import GraphDatabase
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_neo4j import Neo4jChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# --- 1. CONFIGURATION ---
# Use the ABSOLUTE path to your vector database
CHROMA_PATH = r"C:\Users\shreyas\Aerothon\expirements\db_root\public\Knowledge_vectors"
MODEL_NAME = "llama3.1:8b-instruct-q4_K_M"

# Standard local Ollama port is 11434
OLLAMA_BASE_URL = "http://127.0.0.1:11434"

# Neo4j Credentials
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_AUTH = ("neo4j", "password")

# Initialize Local AI Components with explicit base_url to fix ResponseError
llm = ChatOllama(
    model=MODEL_NAME, 
    temperature=0, 
    num_ctx=8192, 
    base_url=OLLAMA_BASE_URL
)
embeddings = OllamaEmbeddings(
    model="nomic-embed-text", 
    base_url=OLLAMA_BASE_URL
)

# --- 2. SILENCE WARNINGS (Schema Initialization) ---
def initialize_neo4j_schema():
    """Defines relationship types in Neo4j to prevent 'UnknownRelationship' warnings."""
    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    with driver.session() as session:
        session.run("""
            MERGE (s:Session {id: 'schema_init'})
            MERGE (m:Message {content: 'init', role: 'system'})
            MERGE (s)-[:LAST_MESSAGE]->(m)
            MERGE (m)-[:NEXT]->(m)
            DETACH DELETE s, m
        """)
    driver.close()

initialize_neo4j_schema()

# --- 3. CONNECT TO DATABASES ---
vectorstore = Chroma(
    persist_directory=CHROMA_PATH, 
    embedding_function=embeddings,
    collection_name="Knowledge_Store"  # Must match your ingestion script
)
# Search k=5 chunks (smaller chunks = more relevant segments)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# --- 4. PROMPTS & LOGIC ---
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given the history, rephrase the user's last question into a standalone question."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a professional Document Analyst. Use the provided context to answer questions precisely or provide comprehensive summaries.

    GUIDELINES:
    1. FACT RETRIEVAL: If the user asks for a specific fact, provide a direct answer grounded in the context.
    2. SUMMARIZATION: If the user asks for a summary (e.g., 'Summarize Chapter 2'), synthesize all relevant details from the provided chunks into a structured overview. Use bullet points for key sub-topics.
    3. INTEGRITY: Use ONLY the provided context. If the context does not contain the information requested, state that the materials are insufficient.
    4. CITATION: Always mention which document or section your information comes from if available.

    Context:
    {context}"""),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

def get_query(input_data):
    if input_data.get("chat_history") and len(input_data["chat_history"]) > 0:
        chain = context_q_prompt | llm | StrOutputParser()
        return chain.invoke(input_data)
    return input_data["input"]

# The core RAG Chain
rag_chain = (
    RunnablePassthrough.assign(
        context=RunnableLambda(get_query) | retriever | format_docs
    )
    | qa_prompt
    | llm
)

# --- 5. THE HYBRID CHAT FUNCTION ---
def chat(user_input: str, session_id: str = "user_session_1"):
    # Connect to persistent history in Neo4j
    history = Neo4jChatMessageHistory(
        url=NEO4J_URI,
        username=NEO4J_AUTH[0],
        password=NEO4J_AUTH[1],
        session_id=session_id
    )
    
    # 1. Generate standalone query based on history
    smart_query = get_query({"input": user_input, "chat_history": history.messages})
    
    # 2. Retrieve most relevant documents
    docs = retriever.invoke(smart_query)
    
    # 3. Generate response using the RAG chain
    response = rag_chain.invoke({"input": user_input, "chat_history": history.messages})
    
    # 4. Update memory in Neo4j
    history.add_user_message(user_input)
    history.add_ai_message(response.content)
    
    # 5. Extract metadata for citations
    sources = []
    for doc in docs:
        src = doc.metadata.get("source", "Unknown")
        pg = doc.metadata.get("page", "N/A")
        sources.append(f"- {src} (Page: {pg})")
    
    return f"{response.content}\n\n**Sources:**\n" + "\n".join(set(sources))

# --- TEST ---
if __name__ == "__main__":
    print(chat("Explain coercive practice with reference to procurement.?"))

According to the provided context, **"Coercive Practice"** refers to any coercion or threat that impairs or harms a party or its property to influence the procurement process or affect the execution of a contract.

In the context of procurement, coercive practices can take many forms, including:

* Threats to withhold payment or benefits
* Threats to cancel contracts or terminate agreements
* Physical intimidation or harassment
* Economic coercion, such as threats to harm a party's business or reputation

Coercive practices are considered unethical and may be in violation of procurement regulations. They can undermine the integrity of the procurement process and create an unfair advantage for one party over another.

**Reference:** The text states: "**“Coercive practice”**: any coercion or any threat to impair or harm, directly or indirectly, any party or its property to influence the procurement process or affect the execution of a contract."

**Sources:**
- 1.DOE-Manual-for-Procureme

In [6]:
import os
from typing import List
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# --- 1. CONFIGURATION ---
# Updated to your new secure_DB folder
CHROMA_PATH = r"C:\Users\shreyas\Aerothon\expirements\secure_DB\public\Knowledge_vectors"
MODEL_NAME = "llama3.1:8b-instruct-q4_K_M"
OLLAMA_BASE_URL = "http://127.0.0.1:11434"

# Initialize Local AI Components
llm = ChatOllama(
    model=MODEL_NAME, 
    temperature=0, 
    num_ctx=8192, 
    base_url=OLLAMA_BASE_URL
)
embeddings = OllamaEmbeddings(
    model="nomic-embed-text", 
    base_url=OLLAMA_BASE_URL
)

# --- 2. CONNECT TO VECTOR STORE ---
vectorstore = Chroma(
    persist_directory=CHROMA_PATH, 
    embedding_function=embeddings,
    collection_name="Knowledge_Store"
)
# Search k=7 chunks for better summary coverage
retriever = vectorstore.as_retriever(search_kwargs={"k": 7})

# --- 3. PROMPTS & LOGIC ---
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given the history, rephrase the user's last question into a standalone question."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a professional Document Analyst. Use the provided context to answer questions precisely or provide comprehensive summaries.

    GUIDELINES:
    1. FACT RETRIEVAL: Provide direct answers grounded in the context.
    2. SUMMARIZATION: Synthesize relevant details into a structured overview. Use bullet points for sub-topics.
    3. INTEGRITY: Use ONLY the provided context. If information is missing, state that the materials are insufficient.
    4. CITATION: Mention source and page numbers.

    Context:
    {context}"""),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

def get_query(input_data):
    if input_data.get("chat_history") and len(input_data["chat_history"]) > 0:
        chain = context_q_prompt | llm | StrOutputParser()
        return chain.invoke(input_data)
    return input_data["input"]

# The core RAG Chain
rag_chain = (
    RunnablePassthrough.assign(
        context=RunnableLambda(get_query) | retriever | format_docs
    )
    | qa_prompt
    | llm
)

# --- 4. THE CHAT FUNCTION ---
# Using a local list instead of Neo4j
chat_history = [] 

def chat(user_input: str):
    # 1. Generate standalone query based on history
    smart_query = get_query({"input": user_input, "chat_history": chat_history})
    
    # 2. Retrieve most relevant documents
    docs = retriever.invoke(smart_query)
    
    # 3. Generate response
    response = rag_chain.invoke({"input": user_input, "chat_history": chat_history})
    
    # 4. Extract metadata
    sources = []
    for doc in docs:
        src = doc.metadata.get("source", "Unknown")
        pg = doc.metadata.get("page", "N/A")
        sources.append(f"- {src} (Page: {pg})")
    
    # 5. Update local memory
    chat_history.extend([
        HumanMessage(content=user_input), 
        AIMessage(content=response.content)
    ])
    
    return f"{response.content}\n\n**Sources:**\n" + "\n".join(set(sources))

# --- TEST ---
if __name__ == "__main__":
    query = "Explain coercive practice with reference to procurement.?"
    print(chat(query))

Unfortunately, I don't have any specific text or guidelines related to "coercive practice" in the context of procurement. However, based on general knowledge, I can provide a summary of what coercive practices might entail in a procurement setting.

**Coercive Practices in Procurement:**

Coercive practices refer to actions that use pressure, intimidation, or manipulation to influence the outcome of a procurement process. These practices can compromise the integrity and fairness of the procurement process, potentially leading to biased decisions.

Some examples of coercive practices in procurement include:

* **Undue influence**: Using personal relationships, favors, or other forms of leverage to sway decision-makers.
* **Bribery**: Offering or accepting bribes to secure a contract or favorably influence the outcome of a procurement process.
* **Unfair treatment**: Failing to provide equal opportunities for all bidders or treating certain suppliers unfairly.

**Key Considerations:**

T